In [19]:
from pathlib import Path
import pandas as pd
import numpy as np
import pyarrow
import regex

# Transaction Preprocessing

## E-transfer Transaction Features
- `e_at_risk`
    - General indicator
    - Probability that a message references a species at risk, or parts thereof
    - *could also be a binary flag*
    - *can look at CITES for this list*
- `e_cad_at_risk`
    - Export indicator
    - Probability that message references a species at risk in Canada, or parts thereof (bear, geese, etc.)
    - *could also be a binary flag*
    - *get animal list from fintrac doc and papers*
    - Maaz: I don't think e_at_risk as defined above is useful. CITES list is very long. I will just focus on the Canadian at-risk. And I will call it e_at_risk.
        - VVCP12477508, BMIY44394974, ZGAN21196261, MXBU42708741, VYOC17525287, XGAL12621643 -- for sure
        - IBCA52577297 -- 'moose meat'. David Wilkerson sent 978 to Steven Cruz. Wilkerson is also the guy who bought the bear fangs and skin. idk what to write for this.
- `e_role`
    - General indicator
    - Probability that a message references an IWT related role (e.g. poacher, coordinator, supplier, breeder, trader)
    - *could also be a binary flag*
    - Maaz: regex search for these words did not yield anything, nor did semantic search for poacher, nor semantic search for breeder
- `e_trad_med`
    - Import indicator
    - Probability that a message references traditional medicine
    - *could also be a binary flag*
    - Maaz: semantic search for traditional medicine only brought up one possible hit: lion's mane. But, this is a plant. So idk.
 

*Do we want to separate these into TO and FROM?*

In [20]:
DATAPATH = Path('../data')
EMTPATH = DATAPATH / 'processed' / 'emt.parquet'
emt_df = pd.read_parquet(EMTPATH)

emt_df['trxn_message'] = emt_df['trxn_message'].fillna('')

In [21]:
# function takes iterable of messages and returns list of if its about animal trafficking or no
def classify_message_animal_trafficking(messages):
    messages = pd.Series(messages) # to use str contains

    keywords = [
        "bears?", 
        "cougars?", 
        "goose", 
        "geese", 
        "lynx", 
        "moose", 
        "crab?", 
        "eel", 
        "elver", 
        "lobsters?", 
        "turtles?", 
        "sharks?", 
        "wolf", 
        "wolves",
        "ivory", 
        "tusks?", 
        "scales?", 
        "fins?", 
        "tortoise", 
        "gecko", 
        "salamander", 
        "pangolin",
        "musk deer",
        "macaque",
        "gall",
        "bile",
        "gall ?stones?",
        "tiger",
        "leopard",
        "bones?",
        "rhinoceros",
        "rhino",
        "shell",
        "seahorses?",
        "alligator",
        "crocodile",
        "horns?",
        "skin",
        "fangs?"
    ]
    pattern = '|'.join(['\\b{}\\b'.format(k) for k in keywords])

    negatives = [
        'shark week',
        'shark ?skin',
        'shark tank',
        'lone wolf',
        'bear grylls',
        'fortnite',
        'fornite',
        'snake ?skin',
        'drank ur bear',
        'wolf-brand',
        'bear market inves',
        'wolf of wall street',
        'electric eel',
        'goose island',
        'shark vacuum',
        'lobster dinner',
        'lobster fest',
        'crab shack',
        'bulldog skin care',
        'turtle wax',
        'goose down',
        'phone skin',
        'fitness club',
        'oil change',
        'duck duck goose',
        'kayak',
        'king crab',
        'soap',
        'phone case',
        'pizzeria',
        'kettle',
        'steakhouse',
        'leopard print',
        'art',
        'tiger balm',
        'leopard spot',
        'tiger woods',
        'tiger king',
        'screen protector',
        'donation',
        'seahorse earrings',
        'tickets',
        'power bank',
        'beach towel',
        'diving service',
        'bedliner',
        'camera'
    ]
    negative_pattern = '|'.join(['{}'.format(k) for k in negatives])

    classification = (messages.str.contains(pattern, case=False, na=False)) & (~messages.str.contains(negative_pattern, case=False, na=False))
    return classification.astype(int)

In [22]:
# fuzzy matching with regex example
# regexpr = regex.compile(r"(\bbear\b){e<=1}|(\bfox\b){e<=1}")
# emt_df['e_at_risk'] = emt_df['trxn_message'].map(lambda m : bool(regexpr.search(m)))

# if you want to implement this in the function I wrote, just change how `pattern` is constructed
# pattern = regex.compile('|'.join(['(\\b{}\\b){{e<=1}}'.format(k) for k in keywords]))

# one challenge is e.g. fox would match to for, which gets you lots of false positives
# read https://pypi.org/project/regex/ and you can see that you can specify which character sets to allow for the fuzzy match
# e.g., {s<=2:[a-z]} at most 2 substitutions, which must be in the character set [a-z]
# so to fuzzy match fox but not for, use something like that to exclude matching the `r` character?

# and of course would have to change the str.contains to a map and lambda function

In [23]:
emt_df['e_at_risk'] = classify_message_animal_trafficking(emt_df['trxn_message'])

In [24]:
# function takes iterable of messages and returns list of if its about a traditional medicine plant or not
# I have left this here but I am not sure if this is useful. the animals are already covered in the previous function.
# based on: https://www.canada.ca/en/environment-climate-change/services/convention-international-trade-endangered-species/publications/animal-plant-ingredients-traditional-medicine.html#_03
def classify_message_trad_medicine_plant(messages):
    messages = pd.Series(messages) # to use str contains

    keywords = [
        "aucklandia",
        "costus",
        "root",
        "eagle ?wood",
        "fern",
        "golden haired dog",
        "golden hair dog",
        "nard"
    ]
    pattern = '|'.join(['\\b{}\\b'.format(k) for k in keywords])

    negatives = []
    negative_pattern = '|'.join(['{}'.format(k) for k in negatives])

    classification = (messages.str.contains(pattern, case=False, na=False)) & (~messages.str.contains(negative_pattern, case=False, na=False))

    return classification

In [25]:
# dummy
def classify_messsage_iwt_role(messages):
    messages = pd.Series(messages)

    classification = [0 for m in messages]

    return classification

In [26]:
emt_df['e_role'] = classify_messsage_iwt_role(emt_df['trxn_message'])

In [27]:
# dummy
def classify_messsage_trad_med(messages):
    messages = pd.Series(messages)

    classification = [0 for m in messages]

    return classification

In [28]:
emt_df['e_trad_med'] = classify_messsage_trad_med(emt_df['trxn_message'])

In [29]:
emt_df.to_parquet(EMTPATH, index=False)
emt_df.sample(3)

,cust_id_sender,cust_id_receiver,name_sender,name_receiver,trxn_message,emt_value,trxn_id,occ_wealth_receiver,occ_animal_receiver,occ_int_receiver,label_receiver,occ_wealth_sender,occ_animal_sender,occ_int_sender,label_sender,e_at_risk,e_role,e_trad_med
297742,EXTERNAL276243,CUST12292844,NICOLE KING,MICHELLE SHEPPARD,,1620.0,ZMVS81936525,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0,0,0
194207,CUST38727749,CUST71060211,FRANÇOISE LAPIERRE,CARLA ADÁN CABALLERO,,100.0,QUUK78960914,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0
251060,EXTERNAL347406,CUST87784549,CARLOTA MIRANDA URBINA,MRS. CYNTHIA THOMAS,,450.0,QWSV26739075,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0,0,0


## Wire Transfer Transaction Features
- `w_to_country`
    - Import indicator
    - Binary flag
    - 1 if the wire transfer is to a jurisdiction of concern:
        - China
        - Hong Kong
        - South Africa
        - Australia
        - ...
- `w_from_country`
    - *not listed in fintrac doc, but could be a useful flag as well*
    - Binary flag
    - 1 if from a jurisdiction of concern, 0 otherwise
- `w_external_to_animal`
    - Binary flag
    - 1 if from a jurisdiction of concern to an animal related business, 0 otherwise

In [31]:
WIREPATH = DATAPATH / 'processed' / 'wire.parquet'
KYCPATH = DATAPATH / 'processed' / 'kyc.parquet'
EMTPATH = DATAPATH / 'processed' / 'emt.parquet'

SUSCOUNTRIES = ['CN', 'AU']

In [32]:
#Load Data
kyc_df = pd.read_parquet(KYCPATH)
kyc_df = kyc_df[['cust_id', 'occ_animal']]

wire_df = pd.read_parquet(WIREPATH)

#w_to_country
wire_df['w_to_country'] = wire_df['country_receiver'].map(lambda x: 1 if x in SUSCOUNTRIES else 0)

#w_from_country
wire_df['w_from_country'] = wire_df['country_sender'].map(lambda x: 1 if x in SUSCOUNTRIES else 0)

#w_external_to_animal
wire_df['w_external_to_animal'] = wire_df.apply(lambda r: 1 if r.country_sender!='CA' and r.occ_animal_receiver==1 else 0, axis=1)

In [33]:
wire_df.to_parquet(WIREPATH, index=False)
wire_df.sample(3)

,cust_id_sender,cust_id_receiver,name_sender,name_receiver,trxn_value,country_sender,country_receiver,trxn_id,occ_wealth_receiver,occ_animal_receiver,occ_int_receiver,label_receiver,occ_wealth_sender,occ_animal_sender,occ_int_sender,label_sender,w_to_country,w_from_country,w_external_to_animal
14524,EXTERNAL357400,CUST21461232,DANA JOHNSON,WENDY TAPIA,2639.0,US,CA,ZDVR74404690,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,0,0,0
20072,CUST53185470,CUST97044980,JENNIFER HORTON,HENRY STEWART,3025.0,CA,CA,LTCI25214358,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0,0,0
3751,EXTERNAL395298,CUST86384621,JOEL JACKSON,TERESA HEBERT,2519.5,CA,CA,YIND86942786,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0,0,0


## Cash Transaction Features
- `c_large`
    - General indicator
    - Binary flag
    - 1 if a deposit or withdrawal is an outlier (above Q3+1.5*IQR)
    - To determine outliers, data is grouped by occupation and by transaction type (deposit/withdrawal)

In [34]:
DATAPATH = Path('../data/')
CASHPATH = DATAPATH / 'processed' / 'cash.parquet'
KYCPATH = DATAPATH / 'processed' / 'kyc.parquet'

UPPERBOUND = 1.5 #Q3 + UPPERBOUND*IQR gives the transaction amount beyond which transactions are classified as outliers

In [35]:
#Load Data
kyc_df = pd.read_parquet(KYCPATH)
kyc_df = kyc_df[['cust_id', 'occupation']]

cash_df = pd.read_parquet(CASHPATH)
cash_df = cash_df.merge(kyc_df, on='cust_id', how='left')

#Get IQR for Outlier Test
q3 = cash_df.groupby(['type', 'occupation'])['trxn_amount'].quantile(0.75)
q1 = cash_df.groupby(['type', 'occupation'])['trxn_amount'].quantile(0.25)
iqr = q3-q1
fence = q3+UPPERBOUND*iqr #upper bound for outlier tests
fence_dict = fence.to_dict()

#Test outliers
def outlier_test(row, fence): 
    outlier = row.trxn_amount > fence[(row.type, row.occupation)]

    return 1 if outlier else 0

cash_df['c_large'] = cash_df.apply(lambda row: outlier_test(row, fence_dict), axis=1)
print(f'Classified {cash_df.c_large.sum()} of {cash_df.c_large.count()} as outliers')

Classified 9774 of 212532 as outliers


In [36]:
cash_df.to_parquet(CASHPATH, index=False)
cash_df.sample(3)

,cust_id,trxn_amount,type,trxn_id,occ_wealth,occ_animal,occ_int,label,occupation,c_large
178714,CUST56157392,7345,deposit,VPFY14245166,0,0,0,0,"Cash-intensive Business Owner (e.g., Laundromat)",0
11199,CUST38536568,1235,withdrawal,PUUZ53118281,0,0,0,0,Nanny,0
128577,CUST46992708,3160,deposit,JNNM59377986,1,0,0,0,Sports Agent,0


# General Transaction Features
- `t_to_animal`
    - Import indicator
    - Binary flag
    - 1 if the transaction is to an animal related business
- `t_from_animal`
    - *not listed in fintrac doc, but could be a useful flag as well*
    - Binary flag
    - 1 if the transaction is from an animal related business
- `t_to_animal_large`
    - Import indicator
    - Binary flag
    - 1 if the transaction is from an animal related and is above a certain dollar amount, 0 otherwise
    - *tbd the dollar amount*
- `t_from_animal_large`
    - *not listed in fintrac doc, but could be a useful flag as well*
    - Binary flag
    - 1 if the transaction is from an animal related and is above a certain dollar amount, 0 otherwise
    - *tbd the dollar amount*
- `t_to_shipping`
    - Export indicator
    - Binary flag
    - 1 if the transaction is to someone in shipping/postal/cargo
    - *no label for this type of occupation yet*


In [44]:
DATAPATH = Path('../data/')
KYCPATH = DATAPATH / 'processed' / 'kyc.parquet'
EMTPATH = DATAPATH / 'processed' / 'emt.parquet'
WIREPATH = DATAPATH / 'processed' / 'wire.parquet'

In [45]:
emt_data = pd.read_parquet(EMTPATH)
wire_data = pd.read_parquet(WIREPATH)
kyc_data = pd.read_parquet(KYCPATH)

threshold = 1000

In [46]:
# Add a new column to each dataframe to indicate the type of transaction
# cash_data['trxn_type'] = 'cash'
emt_data['trxn_type'] = 'emt'
wire_data['trxn_type'] = 'wire'

cols = ['gender', 'occupation', 'age', 'tenure', 'cust_id', 'occ_wealth', 'occ_animal', 'occ_int']

emt_data['t_to_animal'] = np.where((emt_data['occ_animal_receiver']==1), 1, 0)
wire_data['t_to_animal'] = np.where((wire_data['occ_animal_receiver']==1), 1, 0)
# cash_data['t_to_animal'] = np.where(cash_data['occ_animal']==1, 1, 0)

emt_data['t_from_animal'] = np.where((emt_data['occ_animal_sender']==1), 1, 0)
wire_data['t_from_animal'] = np.where((wire_data['occ_animal_sender']==1), 1, 0)
# cash_data['t_from_animal'] = 0

emt_data['t_to_animal_large'] = np.where((emt_data['occ_animal_receiver']==1)  & (emt_data['emt_value'] > threshold), 1, 0)
wire_data['t_to_animal_large'] = np.where((wire_data['occ_animal_receiver']==1)  & (wire_data['trxn_value'] > threshold), 1, 0)
# cash_data['t_to_animal_large'] = np.where((cash_data['occ_animal']==1) & (cash_data['trxn_amount'] > threshold), 1, 0)

emt_data['t_from_animal_large'] = np.where((emt_data['occ_animal_sender']==1) & (emt_data['emt_value'] > threshold), 1, 0)
wire_data['t_from_animal_large'] = np.where((wire_data['occ_animal_sender']==1) & (wire_data['trxn_value'] > threshold), 1, 0)
# cash_data['t_from_animal_large'] = 0

emt_data['t_to_int'] = np.where((emt_data['occ_int_receiver']==1), 1, 0)
wire_data['t_to_int'] = np.where((wire_data['occ_int_receiver']==1), 1, 0)
# cash_data['t_to_int'] = np.where(cash_data['occ_int']==1, 1, 0)

emt_data['t_from_int'] = np.where((emt_data['occ_int_sender']==1), 1, 0)
wire_data['t_from_int'] = np.where((wire_data['occ_int_sender']==1), 1, 0)
# cash_data['t_from_int'] = 0

In [48]:
emt_data.to_parquet('../data/processed/emt.parquet')
wire_data.to_parquet('../data/processed/wire.parquet')